In [ ]:
import os
from docx import Document
from pydub import AudioSegment
import whisper

In [ ]:
# Cargar el archivo de audio original
audio = AudioSegment.from_file("audio/audio.mp3")
carpeta_entrada = "data"
carpeta_salida = "out"
duracion_segmento_ms = 10 * 60 * 1000  # 5 minutos en milisegundos

In [ ]:
# Crear el modelo Whisper
model = whisper.load_model("medium", device="cuda")

# Si no tienes GPU descomemtar
# model = whisper.load_model("medium", device="cpu")

In [ ]:
# Dividir el archivo de audio en fragmentos mas pequeños
for i in range(0, len(audio), duracion_segmento_ms):
    segmento = audio[i : i + duracion_segmento_ms]
    tiempo_inicio = (
        i // 1000
    )  # Convertir milisegundos a segundos para el tiempo de inicio real
    minutos, segundos = divmod(tiempo_inicio, 60)
    nombre_segmento = f"{carpeta_entrada}/segmento_{i//duracion_segmento_ms}_{minutos:02d}_{segundos:02d}.mp3"
    segmento.export(nombre_segmento, format="mp3")

In [ ]:
# Procesar cada segmento de audio y transcribirlo de forma ordenada
def procesar_y_transcribir(carpeta_entrada, carpeta_salida):
    archivos = sorted(os.listdir(carpeta_entrada))  # Asegura un orden correcto
    for archivo in archivos:
        if archivo.endswith(".mp3"):
            print(f"Procesando: {archivo}")
            ruta_completa = os.path.join(carpeta_entrada, archivo)
            result = model.transcribe(ruta_completa, language="es")
            texto_completo = result["text"]

            # Extraer la marca de tiempo directamente del nombre del archivo
            nombre_sin_extension = os.path.splitext(archivo)[0]
            _, marca_tiempo = nombre_sin_extension.rsplit("_", 2)[0], "_".join(
                nombre_sin_extension.rsplit("_", 2)[1:]
            )
            marca_tiempo_formateada = marca_tiempo.replace(
                "_", ":"
            )  # Convertir a formato de tiempo usual

            nombre_archivo_salida = f"{nombre_sin_extension}.txt"
            ruta_salida = os.path.join(carpeta_salida, nombre_archivo_salida)

            with open(ruta_salida, "w", encoding="utf-8") as file:
                # Incluir la marca de tiempo al inicio del archivo de texto
                file.write(
                    f"Tiempo de inicio: {marca_tiempo_formateada}\n\n{texto_completo}"
                )
            print(f"Transcripción guardada en: {ruta_salida}")


procesar_y_transcribir(carpeta_entrada, carpeta_salida)

In [ ]:
# Exportar a documento de Word
carpeta_salida = "out"
archivos_txt = sorted([f for f in os.listdir(carpeta_salida) if f.endswith(".txt")])

doc = Document()

for archivo in archivos_txt:
    with open(os.path.join(carpeta_salida, archivo), "r", encoding="utf-8") as file:
        contenido = file.read()
        doc.add_paragraph(contenido)
        if archivo != archivos_txt[-1]:
            doc.add_page_break()

nombre_doc_word = "Documento_Transcripcion.docx"
doc.save(nombre_doc_word)